In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import my_stuff

### Parsing

In [2]:
url= 'https://en.wikipedia.org/wiki/London'
url1 = 'https://en.wikipedia.org/wiki/Berlin'
url2= 'https://en.wikipedia.org/wiki/Hamburg'
url3= 'https://en.wikipedia.org/wiki/Munich'


In [3]:
response_l = requests.get(url)
london = BeautifulSoup(response_l.content, 'html.parser')

In [4]:
response_b = requests.get(url1)
berlin = BeautifulSoup(response_b.content, 'html.parser')

In [5]:
response_h = requests.get(url2)
hamburg = BeautifulSoup(response_h.content, 'html.parser')

In [6]:
response_m = requests.get(url3)
munich = BeautifulSoup(response_m.content, 'html.parser')

In [7]:
#print(berlin.prettify())

### Scraping for population

In [8]:
pop_london= london.find_all(class_='infobox-header' )[2]
lon_pop= pop_london.find_next(class_= 'infobox-data').get_text()
lon_pop

'8,799,800[1]'

In [9]:
pop_berlin= berlin.find_all(class_= 'infobox-header')[2]
stat_berlin= pop_berlin.find_next(class_='infobox-data')
ber_pop= stat_berlin.get_text()
ber_pop

'3,755,251'

In [10]:
pop_hamburg= hamburg.find_all(class_= 'infobox-header')[2]
stat_hamburg= pop_hamburg.find_next(class_='infobox-data')
ham_pop= stat_hamburg.get_text()
ham_pop

'1,945,532'

In [11]:
pop_munich= munich.find_all(class_= 'infobox-header')[2]
stat_munich= pop_munich.find_next(class_='infobox-data')
mun_pop=stat_munich.get_text()
mun_pop

'1,512,491'

##### Alternative methods of scraping for population

In [12]:
berlin_population= berlin.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(21) > td')[0].get_text()
berlin_population

'3,755,251'

In [13]:
hamburg_population = hamburg.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(19) > td')[0].get_text()
hamburg_population

'1,945,532'

In [14]:
munich_population= munich.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(21) > td')[0].get_text()
munich_population

'1,512,491'

### Scraping for Longitude and Latitude

##### London

In [15]:
for coords in london.find_all(class_= 'infobox-full-data'):
    if coords.find(string =re.compile ('Coordinate', re.IGNORECASE)):
        lat= coords.find(class_= 'latitude').get_text()
        long = coords.find(class_= 'longitude').get_text()
        print(lat)
        print(long)

51°30′26″N
0°7′39″W


##### Berlin

In [16]:
for coords in berlin.find_all(class_= 'infobox-full-data'):
    if coords.find(string=re.compile('Coordinate', re.IGNORECASE)):
        lat_b = coords.find(class_= 'latitude').get_text()
        long_b = coords.find(class_= 'longitude').get_text()
        print(lat_b)
        print(long_b)

52°31′12″N
13°24′18″E


##### Hamburg

In [17]:
for coords in hamburg.find_all(class_= 'infobox-full-data'):
    if coords.find(string=re.compile('Coordinates', re.IGNORECASE)):
        lat_h= coords.find(class_='latitude').get_text()
        long_h = coords.find(class_='longitude').get_text()
        print(lat_h)
        print(long_h)

53°33′N
10°00′E


##### Munich

In [18]:
for coords in munich.find_all(class_= 'infobox-full-data'):
    if coords.find(string=re.compile('Coordinates', re.IGNORECASE)):
        lat_m= coords.find_next(class_='latitude').get_text()
        long_m = coords.find_next(class_='longitude').get_text()
        print(lat_m)
        print(long_m)

48°08′15″N
11°34′30″E


### Scraping for City

In [19]:
city= london.find('h1').get_text()
city

'London'

In [20]:
city1= berlin.find('h1').get_text()
city1

'Berlin'

In [21]:
city2= hamburg.find('h1').get_text()
city2

'Hamburg'

In [22]:
city3= munich.find('h1').get_text()
city3

'Munich'

### Scraping for Country

In [23]:
country_b= berlin.find_all(class_='infobox-label')[0]
country_= country_b.find_next(class_= 'infobox-data')
Country= country_.get_text()
Country

'Germany'

In [24]:
country_l= london.find_all(class_='infobox-label')[0]
country_= country_l.find_next(class_= 'infobox-data')
Country= country_.get_text()
Country

'United Kingdom'

### Creating Dataframe for Results

##### Manually

In [25]:
df ={
    'city':['Berlin','Hamburg','Munich','London'],
    "population": ['3,755,251', '1,945,532','1,512,491','8,799,800'],
    "latitude":['52°31′12″N','53°33′N', '48°08′15″N','51°30′26″N'], 
    "longitude": ['13°24′18″E','10°00′E','11°34′30″E','0°7′39″W'], 
    "country":['Germany','Germany','Germany', 'United Kingdom']}
pd.DataFrame(df)

,city,population,latitude,longitude,country
0,Berlin,"3,755,251",52°31′12″N,13°24′18″E,Germany
1,Hamburg,"1,945,532",53°33′N,10°00′E,Germany
2,Munich,"1,512,491",48°08′15″N,11°34′30″E,Germany
3,London,"8,799,800",51°30′26″N,0°7′39″W,United Kingdom


##### Looping through

In [26]:
cities = ['berlin','london']
city_info= {'city':[],
          'population':[],
         'latitude':[],
         'longitude':[],
         'country':[]}

for city in cities:
    
    url= 'https://en.wikipedia.org/wiki/'+city
    response = requests.get(url)
    pop_soup = BeautifulSoup(response.content, 'html.parser')
    city = pop_soup.find('h1').get_text()


    for header in pop_soup.find_all(class_='infobox-header'):
         if header.find(string=re.compile("population", re.IGNORECASE)):
            population= header.find_next(class_= 'infobox-data').get_text()
        
    for coords in pop_soup.find_all(class_='infobox-full-data'):
        if coords.find(string=re.compile("Coordinates", re.IGNORECASE)):
            longi = coords.find(class_='longitude').get_text()
                    
    for coords in pop_soup.find_all(class_= 'infobox-full-data'):
        if coords.find(string=re.compile("Coordinates", re.IGNORECASE)):
            lat= coords.find(class_= 'latitude').get_text()
                    
    for label in pop_soup.find_all(class_='infobox-label'):
         if label.find(string=re.compile("country", re.IGNORECASE)):
            country = label.find_next(class_= 'infobox-data').get_text()
                   
    city_info['city'].append(city)
    city_info['population'].append(population)
    city_info['latitude'].append(lat)
    city_info['longitude'].append(longi)
    city_info['country'].append(country)
            
cities_df = pd.DataFrame(city_info)
cities_df['population'] = cities_df['population'].str.replace('[1]','')
cities_df['population'] = cities_df['population'].str.replace(',','')
cities_df["population"] = pd.to_numeric(cities_df["population"])
    

In [27]:
cities_df

,city,population,latitude,longitude,country
0,Berlin,3755251,52°31′12″N,13°24′18″E,Germany
1,London,8799800,51°30′26″N,0°7′39″W,England


### Putting data in a function

In [28]:
def city_pop(city):
    cities_info = {'city':[],
          'population':[],
         'latitude':[],
         'longitude':[],
         'country':[]}
    
    url = "https://en.wikipedia.org/wiki/"+city
    response = requests.get(url)
    soup_city = BeautifulSoup(response.content, 'html.parser')
    city = soup_city.find('h1').get_text()
    
    for header in soup_city.find_all(class_="infobox-header"):
        if header.find(string=re.compile("population", re.IGNORECASE)):
            popu = header.find_next(class_="infobox-data").text

    for label in soup_city.find_all(class_="infobox-label"):
        if label.find(string=re.compile("country", re.IGNORECASE)):
            country = label.find_next(class_="infobox-data").text
    
    for coords in soup_city.find_all(class_="infobox-full-data"):
        if coords.find(string=re.compile("Coordinates", re.IGNORECASE)):
            lati = coords.find_next(class_="latitude").text
            longi = coords.find_next(class_="longitude").text
            
    cities_info['city'].append(city)
    cities_info['population'].append(popu)
    cities_info['latitude'].append(lat)
    cities_info['longitude'].append(longi)
    cities_info['country'].append(country)
    
    cities_df = pd.DataFrame(cities_info)
    cities_df['population'] = cities_df['population'].str.replace('[1]','')
    cities_df['population'] = cities_df['population'].str.replace(',','')
    cities_df["population"] = pd.to_numeric(cities_df["population"])
    return cities_df

### Notebook to SQL

In [29]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password =my_stuff.my_sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [30]:
#cities_df.to_sql('city', if_exists ='append', con =connection_string, index = False)

In [31]:
city_from_sql=pd.read_sql('city', con=connection_string)

In [32]:
city_from_sql

,city_id,city,country,population,latitude,longitude
0,1,Berlin,Germany,3755251,52°31′12″N,13°24′18″E
1,2,London,United Kingdom,8799800,51°30′26″N,0°7′39″W


In [33]:
airport_df = {'ICAO':['EDDB','EGLL'] ,"city_id":[1, 2]}
airport_df= pd.DataFrame(airport_df)

In [34]:
#airport_df.to_sql('airport', if_exists ='append', con =connection_string, index = False)

In [35]:
airport_df = pd.read_sql('airport', con= connection_string)
airport_df

,icao,city_id
0,EDDB,1
1,EGLL,2
